In [1]:
import pandas as pd
import sqlite3
import bz2

url = "http://awing.kis.agh.edu.pl:8080/detectors_names_traffic_s_small.csv.bz2"
file_name = "detectors.csv.bz2"

import urllib.request
urllib.request.urlretrieve(url, file_name)

with bz2.open(file_name, "rt") as f:
    df = pd.read_csv(f)

In [2]:
conn = sqlite3.connect("proj6_readings.sqlite")
df.to_sql("readings", conn, if_exists="replace", index=False)
# conn.close()

50000000

In [3]:
cur = conn.cursor()

cur.execute("""CREATE INDEX detector_id ON readings (detector_id);""")
cur.execute("""CREATE INDEX starttime ON readings (starttime);""")
conn.commit()

In [16]:
ex1 = pd.read_sql("SELECT COUNT(DISTINCT detector_id) FROM readings", conn)
print(ex1.shape)
ex1.to_pickle("proj6_ex01_detector_no.pkl")

(1, 1)


In [21]:
ex2 = pd.read_sql("SELECT detector_id, COUNT(detector_id), MIN(starttime), MAX(starttime) FROM readings GROUP BY detector_id", conn)
ex2.to_pickle("proj6_ex02_detector_stat.pkl")

In [27]:
ex3 = pd.read_sql("""
                    SELECT 
                        detector_id,
                        count,
                        LAG(count) OVER (PARTITION BY detector_id ORDER BY starttime) AS prev_count
                    FROM readings
                    WHERE detector_id = 146
                    ORDER BY starttime
                    LIMIT 500;
                  """, conn)
ex3.to_pickle("proj6_ex03_detector_146_lag.pkl")

In [29]:
ex4 = pd.read_sql("""
                    SELECT 
                        detector_id,
                        starttime,
                        count,
                        SUM(count) OVER (
                            PARTITION BY detector_id 
                            ORDER BY starttime 
                            ROWS BETWEEN CURRENT ROW AND 9 FOLLOWING
                        ) AS rolling_sum
                    FROM readings
                    WHERE detector_id = 146
                    ORDER BY starttime
                    LIMIT 500;
                    """, conn)
ex4.to_pickle("proj6_ex04_detector_146_rolling10.pkl")